In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p00'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/galactic_plane_checks'
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize()
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.0/'

In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, cpus=20):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr

def get_template_tr(template, gamma=2.7, cutoff_tev=np.inf, cpus=20):
    cutoff_gev = cutoff_tev * 1000.
    gp_conf = {
        'template': template,
        'flux': cy.hyp.PowerLawFlux(gamma, energy_cutoff=cutoff_gev),
        'randomize': ['ra'],
        'fitter_args': dict(gamma=gamma),
        'sigsub': True,
        'update_bg': True,
        'fast_weight': False,
    }
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr


#### Get TrialRunners

In [ ]:
tr_dict = {
    'pi0': get_gp_tr('pi0'),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
}

#### Get Results for each template

In [ ]:
res_dict = {}
for key in tr_dict.keys():
    f_path = os.path.join(
        cg.base_dir, 
        'gp/results/{}/{}_unblinded.npy'.format(key, key), 
    )
    res_dict[key] = np.load(f_path)

#### Print best fit fluxes

In [ ]:

dNdE = tr_dict['pi0'].to_dNdE(ns=res_dict['pi0'][1], E0=1e5)
E2dNdE = tr_dict['pi0'].to_E2dNdE(ns=res_dict['pi0'][1], E0=100, unit=1e3)
print(dNdE, E2dNdE)


#### Get bkg fits for each template

In [ ]:
bkg_file_dict = {
    'pi0': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'pi0'),
    'kra5': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra5'),
    'kra50': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra50'),
}
n_bkg_trials = 20000
seed = 1337

bkg_dict = {}
for key, tr in tr_dict.items():
    if key in bkg_file_dict:
        print('Loading background trials for template {}'.format(key))
        sig = np.load(bkg_file_dict[key], allow_pickle=True)
        bkg_dict[key] = sig['poisson']['nsig'][0.0]['ts']
    
    else:
        print('Running background trials for template {}'.format(key))
        bkg_dict[key] = tr.get_many_fits(
            n_trials=n_bkg_trials, seed=seed, mp_cpus=20)
        

#### Plot ts distribution

In [ ]:
for key, bg in bkg_dict.items():
    bg_tsd = cy.dists.TSD(bg)
    fig, ax = plot_bkg_trials(bg_tsd)
    ts = res_dict[key][0]
    ns = res_dict[key][1]
    ax.axvline(
        ts, color='0.8', ls='--', lw=2,
        label='TS: {:3.3f} | ns: {:3.1f}'.format(ts, ns), 
    )
    ts_5sig = bg_tsd.isf_nsigma(5)
    ax.axvline(
        ts_5sig, ls='--', lw=1,
        label='5-sigma TS: {:3.3f}'.format(ts_5sig), 
    )
    ax.set_title('Template: {}'.format(key))
    ax.set_yscale('log')
    ax.legend()
    fig.savefig('{}/ts_dist_{}.png'.format(plot_dir, key))

#### Compute Significance

In [ ]:
p_val_dict = {}
sigma_dict = {}
max_n = 10000000
for key, bg in bkg_dict.items():
    print(key)
    bg_tsd = cy.dists.TSD(bg[:max_n])
    p_val_dict[key] = bg_tsd.sf(bg[:max_n])
    sigma_dict[key] = bg_tsd.sf_nsigma(bg[:max_n])


#### Plot Trial Correlation

In [ ]:
mask = np.zeros_like(bkg_dict['pi0'][:max_n])
sigma_threshold = 0.5

for key, tr in sigma_dict.items():

    mask = np.logical_or(mask, sigma_dict[key] > sigma_threshold)
    

In [ ]:
import matplotlib as mpl

def plot_corr_ax(ax, key1, key2, mask=None, norm=None):
    
    if mask is None:
        mask = np.ones_like(sigma_dict[key1], dtype=bool)
        
    ax.hist2d(
        sigma_dict[key1][mask], sigma_dict[key2][mask],
        bins=bins, norm=norm, cmin=1,
    )
    ax.plot(
        (bins[0][0], bins[0][-1]), (bins[0][0], bins[0][-1]), 
        ls='--', color='0.7', lw=3,
    )
    ax.set_xlabel('$n\cdot \sigma$ of {}'.format(key1))
    ax.set_ylabel('$n\cdot \sigma$ of {}'.format(key2))

fig, axes = plt.subplots(3, 1, figsize=(9, 9))

bins = (np.linspace(0, 6), np.linspace(0, 6))

plot_corr_ax(axes[0], 'pi0', 'kra5', mask=mask)
plot_corr_ax(axes[1], 'kra5', 'kra50', mask=mask)
plot_corr_ax(axes[2], 'pi0', 'kra50', mask=mask)
fig.tight_layout()


In [ ]:
from scipy import stats

corr_keys = ['pi0', 'kra5', 'kra50']

max_nsigma = np.max(
    np.stack([sigma_dict[k] for k in corr_keys]),
    axis=0,
)


fig, ax = plt.subplots()
ax.hist(max_nsigma, bins=np.linspace(0, 6, 1000))
ax.set_xlabel('Max n-sigma')
ax.set_yscale('log')

bg_max = cy.dists.TSD(max_nsigma)



nsigma_chosen = 4.68
pval_chosen = stats.norm.sf(nsigma_chosen)
nsigma_corrected = bg_max.sf_nsigma(nsigma_chosen)

pval_corrected = bg_max.sf(nsigma_chosen)
print('Correcting for: {}'.format(corr_keys))
print('Pre-trial N-sigma of: {}'.format(nsigma_chosen))
print('Post-trial correlated: {} | factor: {}'.format(nsigma_corrected, pval_corrected/pval_chosen))
print('Post-trial conservative: {} | factor: {}'.format(stats.norm.isf(pval_chosen * len(corr_keys)), len(corr_keys)))



In [ ]:
ss_results = np.load(
    os.path.join(
        cg.base_dir, 'skyscan/results/unblinded_skyscan.npy'),
    allow_pickle=True,
)[()]
ss_trial = ss_results['ss_trial']



In [ ]:
ss_results


In [ ]:
# https://www.nasa.gov/mission_pages/GLAST/news/gammaray_best.html
# https://www.nasa.gov/images/content/317870main_Fermi_3_month_labeled_new.jpg
fermi_sources = {
    # ra, dec
    'NGC 1275': (049.9506656698585, +41.5116983765094),
    '3C 454.3': (343.49061658, +16.14821142),
    '47 Tuc': (006.022329, -72.081444),
    '0FGL J1813.5-1248': (273.349033, -12.766842),
    '0FGL J0614.3-3330': (093.5431162, -33.4983656),
    'PKS 0727-115': (112.57963530917, -11.68683347528),
    'Vela': (128.5000, -45.8333),
    'Geminga': (098.475638, +17.770253),
    'Crab': (083.63308, +22.01450),
    'LSI +61 303': (040.1319341179735, +61.2293308716971),
    'PSR J1836+5925': (279.056921, +59.424936),
    'PKS 1502+106': (226.10408242258, +10.49422183753),
    #'Cygnus X-3': (308.10742, +40.95775),
}

cat_dict = {}
for cat_str in ['pwn', 'snr', 'unid']:
    catalog_file = os.path.join(
        cg.catalog_dir, '{}_ESTES_12.pickle'.format(cat_str))
    cat_dict[cat_str] = np.load(catalog_file, allow_pickle=True)


#### Plot template contours

In [ ]:
class ContourSkymap:
    def __init__(self, skymap, nside=None):
        
        # upscale skymap
        if nside is not None:
            skymap = hp.ud_grade(skymap, nside_out=nside)
        else:
            nside = hp.get_nside(skymap)
        
        # normalize such that sum of pixel values equals one
        self.prob_values = skymap / np.sum(skymap)
        self.neg_llh_values = -np.log10(self.prob_values)
        self.nside = hp.get_nside(self.prob_values)
        self.npix = hp.nside2npix(self.nside)
        
        self.theta, self.phi = self.get_healpix_grid()
        
        # sort healpix points according to neg llh
        sorted_indices = np.argsort(self.neg_llh_values)
        self.theta_s = self.theta[sorted_indices]
        self.phi_s = self.phi[sorted_indices]
        self.neg_llh_values_s = self.neg_llh_values[sorted_indices]
        self.prob_values_s = self.prob_values[sorted_indices]
    
        self.cdf_values_s = np.cumsum(self.prob_values_s)
    
    def get_healpix_grid(self):
        npix = hp.nside2npix(self.nside)
        theta, phi = hp.pix2ang(self.nside, np.r_[:npix])
        return theta, phi
    
    def quantile_to_pdf_value(self, quantile):
        """Get pixel probability value
        """
        assert quantile >= 0., quantile
        assert quantile <= 1., quantile

        index = np.searchsorted(self.cdf_values_s, quantile)
        return self.prob_values_s[index]
    
    def _get_level_indices(self, level=0.5, delta=0.01):
        """Get indices of healpix map, which belong to the specified
        contour as defined by: level +- delta.

        Parameters
        ----------
        level : float, optional
            The contour level. Example: a level of 0.7 means that 70% of events
            are within this contour.
        delta : float, optional
            The contour is provided by selecting directions from the sampled
            ones which have cdf values within [level - delta, level + delta].
            The smaller delta, the more accurate the contour will be. However,
            the number of available sample points for the contour will also
            decrease.

        Returns
        -------
        int, int
            The starting and stopping index for a slice of sampled events
            that lie within the contour [level - delta, level + delta].

        Raises
        ------
        ValueError
            If number of resulting samples is too low.
        """
        assert level >= 0., level
        assert level <= 1., level

        index_min = np.searchsorted(self.cdf_values_s, level - delta)
        index_max = min(self.npix,
                        np.searchsorted(self.cdf_values_s, level + delta))

        if index_max - index_min <= 10:
            raise ValueError('Number of samples is too low!')

        return index_min, index_max
    
    def contour(self, level=0.5, delta=0.01):
        """Get zenith/azimuth paris of points that lie with the specified
        contour [level - delta, level + delta].

        Parameters
        ----------
        level : float, optional
            The contour level. Example: a level of 0.7 means that 70% of events
            are within this contour.
        delta : float, optional
            The contour is provided by selecting directions from the sampled
            ones which have cdf values within [level - delta, level + delta].
            The smaller delta, the more accurate the contour will be. However,
            the number of available sample points for the contour will also
            decrease.

        Returns
        -------
        np.array, np.array
            The theta/phi pairs that lie within the contour
            [level - delta, level + delta].
        """
        index_min, index_max = self._get_level_indices(level, delta)
        return (self.theta_s[index_min:index_max],
                self.phi_s[index_min:index_max])

In [ ]:
def get_smeared_template(key, smearing=5):
    space_pdf = tr_dict[key].llh_models[0].pdf_ratio_model.models[0]
    sigma_idx = np.searchsorted(np.rad2deg(space_pdf.sigmas), smearing)
    template_smeared = space_pdf.pdf_space_sig[sigma_idx]
    return np.array(template_smeared)


#### Skymap Plotting Class

In [ ]:
import utils
import contour_compute


class SkymapPlotter:
    
    def __init__(self, fermi_sources=fermi_sources, cat_dict=cat_dict, ss_results=ss_results, **kwargs):
        self.cat_dict = cat_dict
        self.fermi_sources = fermi_sources
        self.ss_results = ss_results
        self.coord = None
        
        self.fig, self.ax, self.sp = self.plot_skymap(**kwargs)
    
    def add_skymap_layer(self, m, ax=None, **kw):
        if ax is None:
            ax = self.ax
        lat, lon, Z = self.sp.map_to_latlonz(m)
        pc = ax.pcolormesh(lon, lat, Z, **kw)
        return pc
        
    @staticmethod
    def plot_skymap(
                skymap, fig=None, ax=None, outfile=None, figsize=(9, 6),
                vmin=None, vmax=None, label=None, norm=None,
                cmap=cy.plotting.skymap_cmap,
                input_coord='C',
                n_cb_ticks=4,
                gp_kw=dict(color='.3', alpha=0.5), gp_lw=1.,
                plot_gp=True, annotate=True,
                **kwargs
            ):
        """Plot a skymap

        Parameters
        ----------
        skymap : array_like
            The skymap to plot.
        outfile : str, optional
            The output file path to which to plot if provided.
        vmin : float, optional
            The minimum value for the colorbar.
        vmax : float, optional
            The maximum value for the colorbar.
        figsize : tuple, optional
            The figure size to use.
        label : str, optional
            The label for the colorbar.

        Returns
        -------
        fig, ax
            The matplotlib figure and axis.
        """
        if fig is None:
            fig, ax = plt.subplots(
                subplot_kw=dict(projection='aitoff'), figsize=figsize)

        if 'coord' in kwargs and kwargs['coord'] == 'G':
            nohr = True
            if input_coord == 'C':
                skymap = SkymapPlotter.equatorial_to_galactic(skymap)
        else:
            nohr = False

        sp = cy.plotting.SkyPlotter(
            pc_kw=dict(cmap=cmap, vmin=vmin, vmax=vmax, norm=norm), 
            **kwargs
        )
        mesh, cb = sp.plot_map(ax, skymap, n_ticks=n_cb_ticks, nohr=nohr)
    
        SkymapPlotter.annotate_skymap(
            ax=ax, sp=sp, annotate=annotate, plot_gp=plot_gp, gp_kw=gp_kw, gp_lw=gp_lw,
        )
        if False:
            if sp.coord == 'G' and annotate:
                kw = dict(xycoords='axes fraction', textcoords='offset pixels', verticalalignment='center')
                ax.annotate(r'l = -180°', xy=(1, .5), xytext=(10, 0), horizontalalignment='left', **kw)
                ax.annotate(r'l = 180°', xy=(0, .5), xytext=(-10, 0), horizontalalignment='right', **kw)

            if sp.coord != 'G' and plot_gp:
                sp.plot_gp(ax, lw=gp_lw, **gp_kw)
                sp.plot_gc(ax, **gp_kw)
            kw = dict(color='.5', alpha=.5)
            ax.grid(**kw)
        cb.set_label(label)
        fig.tight_layout()
        if outfile is not None:
            fig.savefig(outfile)

        return fig, ax, sp
    
    @staticmethod
    def annotate_skymap(ax, sp, annotate=True, plot_gp=True, gp_kw=dict(color='.3', alpha=0.5), gp_lw=1.):
        if sp.coord == 'G' and annotate:
            kw = dict(xycoords='axes fraction', textcoords='offset pixels', verticalalignment='center')
            ax.annotate(r'l = -180°', xy=(1, .5), xytext=(10, 0), horizontalalignment='left', **kw)
            ax.annotate(r'l = 180°', xy=(0, .5), xytext=(-10, 0), horizontalalignment='right', **kw)
        
        if sp.coord != 'G' and plot_gp:
            sp.plot_gp(ax, lw=gp_lw, **gp_kw)
            sp.plot_gc(ax, **gp_kw)
        kw = dict(color='.5', alpha=.5)
        ax.grid(**kw)
    
    @staticmethod
    def equatorial_to_galactic(m, rot=180):
        r = hp.Rotator(rot=rot, coord='CG')
        return r.rotate_map_pixel(m)

    @staticmethod
    def equatorial_to_galactic_coords(theta, phi, rot=180):
        r = hp.Rotator(rot=rot, coord='CG')
        return r(theta, phi)
    
    def convert_theta_phi_to_mpl_coords(self, theta, phi, convert=True):
        if self.sp.coord == 'G' and convert:
            theta, phi = self.equatorial_to_galactic_coords(theta, phi)
        x, y = self.sp.thetaphi_to_mpl(theta, phi)
        return x, y
        
    def convert_ra_dec_to_mpl_coords(self, ra, dec):
        theta = np.pi/2. - dec
        phi = ra
        return self.convert_theta_phi_to_mpl_coords(theta=theta, phi=phi)

    def draw_equator(self, ax=None, color='0.6', s=1, **kwargs):
        if ax is None:
            ax = self.ax
        phi = np.linspace(0., 2*np.pi, 10000)
        dec = np.zeros_like(phi)
        x, y = self.convert_ra_dec_to_mpl_coords(ra=phi, dec=dec)
        return ax.scatter(x, y, marker='.', color=color, s=s)

    def plot_catalog(self, ax=None, sp=None, marker='x', color='red'):
        
        if ax is None:
            ax = self.ax
        if sp is None:
            sp = self.sp
            
        for cat_str, cat in self.cat_dict.items():
            x, y = self.convert_ra_dec_to_mpl_coords(
                ra=np.deg2rad(cat.ra_deg), dec=np.deg2rad(cat.dec_deg))
            ax.scatter(x, y, marker=marker, color=color, label=cat_str)
    
    def plot_hotspots(self, ax=None, sp=None, marker='x', color='0.8', **kwargs):
        if ax is None:
            ax = self.ax
        if sp is None:
            sp = self.sp
            
        # plot hottest spots
        for res_str in ['ipix_max_north', 'ipix_max_south']:
            theta, phi = hp.pix2ang(128, self.ss_results[res_str])
            x, y = self.convert_theta_phi_to_mpl_coords(theta=theta, phi=phi)
            ax.scatter(x, y, marker=marker, color=color, **kwargs)
    
    def plot_fermi_sources(self, ax=None, sp=None, marker='+', color='1.0'):
        
        if ax is None:
            ax = self.ax
        if sp is None:
            sp = self.sp
            
        for key, (ra_deg, dec_deg) in self.fermi_sources.items():
            x, y = self.convert_ra_dec_to_mpl_coords(
                ra=np.deg2rad(ra_deg), dec=np.deg2rad(dec_deg))
            ax.scatter(x, y, marker=marker, color=color)
    
    def get_contour(self, skymap, quantiles=[0.5], geodesic='planar'):
        """Get contours

        Returns
        -------
        contours_by_level : list(list(list(point)))
            The contours for each level
            Outermost list indexes by level
            Second list indexes by contours at a particular level
            Third list indexes by points in each contour
            Points are of the same form as sample_points
        """
        nside = hp.get_nside(skymap)
        theta, phi = hp.pix2ang(nside=nside, ipix=np.arange(hp.nside2npix(nside)))
        
        if self.sp.coord == 'G':
            skymap = SkymapPlotter.equatorial_to_galactic(skymap)
        
        # compute PDF levels for provided quantiles 
        contour_map = ContourSkymap(skymap=skymap)

        levels = []
        for quantile in quantiles:
            levels.append(contour_map.quantile_to_pdf_value(quantile))
        
        # compute sample points in which to compute the contours
        if geodesic == 'spherical':
            sample_points = np.stack((theta, phi), axis=1)
        elif geodesic == 'planar':
            x, y = self.convert_theta_phi_to_mpl_coords(theta=theta, phi=phi, convert=False)
            sample_points = np.stack((x, y), axis=1)
        else:
            raise ValueError('Unknown geodesic: {}'.format(geodesic))

        contours = contour_compute.compute_contours(
            sample_points=sample_points, samples=contour_map.prob_values, levels=levels, geodesic=geodesic)
        return contours

    
    def plot_template_contour(
                self, template_str, smearing_deg=5., quantiles=[0.5], ls=['-'], color=[None], geodesic='planar',
        ):
        assert len(quantiles) == len(ls)
        assert len(quantiles) == len(color)
        
        template = get_smeared_template(template_str, smearing=smearing_deg)
        contours = self.get_contour(skymap=template, quantiles=quantiles, geodesic=geodesic)
        for ls_i, color_i, contour in zip(ls, color, contours):
            for contour_i in contour:
                if geodesic == 'spherical':
                    x, y = self.convert_theta_phi_to_mpl_coords(theta=contour_i[:, 0], phi=contour_i[:, 1], convert=False)
                elif geodesic == 'planar':
                    x, y = contour_i[:, 0], contour_i[:, 1]
                else:
                    raise ValueError('Unknown geodesic: {}'.format(geodesic))
                if len(x) > 2:
                    skymap_plotter.ax.plot(x, y, ls=ls_i, color=color_i)
        return contours
    
    def plot_template_contour_points(
                self, template_str, level, 
                smearing_deg=5, color='0.2', delta=0.01, 
                marker='.', s=1,
                ax=None, sp=None,
            ):
        contour_map = ContourSkymap(get_smeared_template(template_str, smearing=smearing_deg))
        theta, phi = contour_map.contour(level=level, delta=delta)
        x, y = self.convert_theta_phi_to_mpl_coords(theta=theta, phi=phi)
        self.ax.scatter(x, y, marker=marker, s=s, color=color)


#### Plot Gamma

In [ ]:
cmap = plt.cm.viridis  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
#cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(1, 4, 17)
print('bounds', bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

gamma_masked = np.array(ss_trial[3])
#gamma_masked[10**(-ss_trial[0]) > 0.5] = np.nan


skymap_plotter = SkymapPlotter(skymap=gamma_masked, cmap=cmap, norm=norm, label=r'$\gamma$')

if False:
    def get_alpha_cmap(N=100):
        cmaplist = [(1., 1., 1., (N-i-1)/N) for i in range(N)]

        # create the new map
        cmap = mpl.colors.LinearSegmentedColormap.from_list(
            'Custom cmap', cmaplist, N)

        # define the bins and normalize
        bounds = np.linspace(-4, 3., 101)
        norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

        return cmap, norm
    
    ss_nsigmas = stats.norm.isf(10**-ss_trial[0])
    cmap_w_alpha, norm_w_alpha = get_alpha_cmap()
    for s in np.linspace(-2, 4, 10):
        print(s, cmap_w_alpha(norm_w_alpha(s)))
    skymap_plotter.add_skymap_layer(
        ss_nsigmas, cmap=cmap_w_alpha, norm=norm_w_alpha,
    )
    # redraw grid and annotations
    skymap_plotter.annotate_skymap(ax=skymap_plotter.ax, sp=skymap_plotter.sp)

#skymap_plotter.plot_catalog()
skymap_plotter.plot_hotspots()
#skymap_plotter.plot_fermi_sources()

skymap_plotter.fig.savefig('{}/skymap_gamma.png'.format(plot_dir))


#### Plot Gamma Alpha

In [ ]:
from scipy import stats

def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

def get_alpha_cmap(alpha_or_amount=1., is_alpha=True, base=plt.cm.viridis):
    cmap = base  # define the colormap
    # extract all colors from the .jet map
    _cmaplist = [cmap(i) for i in range(cmap.N)]
    # force the first color entry to be grey
    #_cmaplist[0] = (.5, .5, .5, 1.0)
    
    # force alpha
    if is_alpha:
        cmaplist = []
        for cmap_item in _cmaplist:
            cmap_item = [c for c in cmap_item]
            cmap_item[3] = alpha_or_amount
            cmaplist.append(cmap_item)
    else:
        cmaplist = [lighten_color(c, amount=alpha_or_amount) for c in _cmaplist]

    # create the new map
    cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist, cmap.N)

    # define the bins and normalize
    bounds = np.linspace(1, 4, 17)
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    
    return cmap, norm

cmap, norm = get_alpha_cmap(alpha_or_amount=1.)
nans = np.ones_like(ss_trial[3]) * np.nan
skymap_plotter = SkymapPlotter(skymap=nans, cmap=cmap, norm=norm, label=r'$\gamma$')
skymap_plotter.add_skymap_layer(
    gamma_masked,
    cmap=cmap, norm=norm,
)

ss_nsigmas = stats.norm.isf(10**-ss_trial[0])

sigma_bins = np.linspace(0, 3, 9)
#pval_bins = np.logspace(0, -5, 12)
#print('pval_bins', pval_bins)
#sigma_bins = pval_bins
sigma_idx = np.digitize(ss_nsigmas, bins=sigma_bins)

for i, sigma in enumerate(sigma_bins):
    mask = sigma_idx == i
    #mask = ss_nsigmas > sigma
    #mask = 10**-ss_trial[0] < sigma
    mask_pole = ss_trial[3] == 0.

    gamma_masked = np.array(ss_trial[3])
    gamma_masked[~mask] = np.nan
    gamma_masked[mask_pole] = np.nan
    
    if np.sum(np.isfinite(gamma_masked)) == 0:
        continue
        
    #alpha = (sigma - sigma_bins[0]) / (sigma_bins[-1] - sigma_bins[0])
    alpha = ((sigma - 0) / (sigma_bins[-1] - 0))**2
    alpha = np.clip(alpha, 0., 1.)
    #alpha = 1./len(sigma_bins) * 0.1
    print(i, sigma, alpha, np.sum(np.isfinite(gamma_masked)))
    cmap, norm = get_alpha_cmap(alpha_or_amount=alpha, is_alpha=True)
    
    skymap_plotter.add_skymap_layer(
        gamma_masked,
        cmap=cmap, norm=norm,
    )

# redraw grid and annotations
skymap_plotter.annotate_skymap(ax=skymap_plotter.ax, sp=skymap_plotter.sp)

#skymap_plotter.plot_catalog()
skymap_plotter.plot_hotspots()
#skymap_plotter.plot_fermi_sources()

skymap_plotter.fig.savefig('{}/skymap_gamma_alpha.png'.format(plot_dir))


#### Plot ns

In [ ]:
skymap_plotter = SkymapPlotter(skymap=ss_trial[2], cmap='viridis', label='ns', vmin=0, vmax=None)
#skymap_plotter.plot_catalog()
skymap_plotter.plot_hotspots()
#skymap_plotter.plot_fermi_sources()

skymap_plotter.fig.savefig('{}/skymap_ns.png'.format(plot_dir))
    

#### Plot p-value plot

In [ ]:
skymap_plotter = SkymapPlotter(
    skymap=ss_trial[0], label=r'$-\log_{10}(p_\mathrm{local})$', vmin=0, vmax=6, 
    gp_kw=dict(color='0.3', alpha=0.5), n_cb_ticks=2, gp_lw=1.)
#skymap_plotter.plot_catalog(marker='x', color='k')
skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
skymap_plotter.fig.savefig('{}/skymap_pvalues.png'.format(plot_dir))


#### Plot N-sigmas

In [ ]:
from scipy import stats

ss_nsigmas =  stats.norm.isf(10**-ss_trial[0])

min_val = -0.52
mask = ss_nsigmas <= min_val
ss_nsigmas[mask] = min_val

mask_pole = ss_trial[3] == 0.
ss_nsigmas[mask_pole] = np.nan

cmap = plt.cm.RdBu_r  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
#cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(-4, 4, 18)
print('bounds', bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)


skymap_plotter = SkymapPlotter(
    skymap=ss_nsigmas, cmap=cmap, norm=norm, label=r'$n\cdot \sigma$', 
    gp_kw=dict(color='0.3', alpha=0.5), gp_lw=1., n_cb_ticks=9)
#skymap_plotter.plot_catalog(marker='x', color='k')
skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
#skymap_plotter.plot_fermi_sources(marker='+', color='green')

smearing_deg = 7
q = 0.2
#skymap_plotter.plot_template_contour('pi0', quantiles=[q], color=['0.2'], ls=['--'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('kra5', quantiles=[q], color=['green'], ls=['--'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('pi0', quantiles=[q, 0.5], color=['0.2', '0.5'], ls=['--', '-.'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour_points('pi0', level=q, color='0.2', smearing_deg=smearing_deg, delta=0.007)
#skymap_plotter.plot_template_contour_points('kra5', level=q, color='green', smearing_deg=smearing_deg, delta=0.004)

#skymap_plotter.fig.savefig('{}/skymap_nsigma_s{:1.1f}_q{:1.1f}.png'.format(plot_dir, smearing_deg, q))
skymap_plotter.fig.savefig('{}/skymap_nsigma.png'.format(plot_dir, smearing_deg, q))


In [ ]:
plt.hist(ss_nsigmas, bins=np.linspace(-1, 0, 1000))
plt.axvline(-0.52, color='0.7')

#### Skymap in Galactic Coordinates: nsigmas

In [ ]:
from scipy import stats

ss_nsigmas =  stats.norm.isf(10**-ss_trial[0])
mask_pole = ss_trial[3] == 0.
ss_nsigmas[mask_pole] = np.nan

cmap = plt.cm.RdBu_r  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
#cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(-4, 4, 18)
print('bounds', bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

skymap_plotter = SkymapPlotter(skymap=ss_nsigmas, cmap=cmap, norm=norm, label=r'$n\cdot \sigma$', coord='G')
#skymap_plotter.plot_catalog(marker='x', color='k')
skymap_plotter.plot_hotspots(marker='x', color='0.8')
#skymap_plotter.plot_fermi_sources(marker='+', color='green')

smearing_deg = 5
q = 0.5
skymap_plotter.plot_template_contour('pi0', quantiles=[q], color=['0.2'], ls=['--'], smearing_deg=smearing_deg)
skymap_plotter.plot_template_contour('kra5', quantiles=[q], color=['green'], ls=['--'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('pi0', quantiles=[q, 0.5], color=['0.2', '0.5'], ls=['--', '-.'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour('kra5', quantiles=[q, 0.5], color=['green', 'green'], ls=['--', '-.'], smearing_deg=smearing_deg)
#skymap_plotter.plot_template_contour_points('pi0', level=q, color='0.2', smearing_deg=smearing_deg, delta=0.004)
#skymap_plotter.plot_template_contour_points('kra5', level=q, color='green', smearing_deg=smearing_deg, delta=0.004)
skymap_plotter.draw_equator()

skymap_plotter.fig.savefig('{}/skymap_nsigma_galactic.png'.format(plot_dir))
#skymap_plotter.fig.savefig('{}/skymap_nsigma_galactic_s{:1.1f}_q{:1.1f}.png'.format(plot_dir, smearing_deg, q))


#### Skymap in Galactic Coordinates: p-value

In [ ]:
skymap_plotter = SkymapPlotter(
    skymap=ss_trial[0], label=r'$-\log_{10}(p_\mathrm{local})$', vmin=0, vmax=6, gp_kw=dict(color='0.3', alpha=0.5), gp_lw=1., coord='G')
#skymap_plotter.plot_catalog(marker='x', color='k')
skymap_plotter.plot_hotspots(marker='x', color='0.8', s=20)
skymap_plotter.draw_equator(color='0.6', s=0.005)
skymap_plotter.fig.savefig('{}/skymap_pvalues_galactic.png'.format(plot_dir))


#### Skymap in Galactic Coordinates: gamma

In [ ]:
cmap = plt.cm.viridis  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
#cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(1, 4, 17)
print('bounds', bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

gamma_masked = np.array(ss_trial[3])
gamma_masked[10**(-ss_trial[0]) > 0.5] = np.nan


skymap_plotter = SkymapPlotter(skymap=gamma_masked, cmap=cmap, norm=norm, label=r'$\gamma$', coord='G')
#skymap_plotter.plot_catalog(marker='x', color='k')
skymap_plotter.plot_hotspots(marker='x', color='0.8')
#skymap_plotter.plot_fermi_sources(marker='+', color='green')
#skymap_plotter.plot_template_contour('pi0', level=0.5, color='0.2', smearing_deg=5)
#skymap_plotter.plot_template_contour('kra5', level=0.5, color='green', smearing_deg=5)
skymap_plotter.draw_equator()


# alpha
if False:
    # create alpha colors
    cmaplist = [(0., 0., 0., (cmap.N - float(i))/cmap.N) for i in range(cmap.N)]

    # create the new map
    cmap_alpha = mpl.colors.LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist, cmap.N)
    norm_alpha = mpl.colors.BoundaryNorm(np.linspace(-5, 5, 17), cmap.N)
    sp.plot_map(ax, m=ss_nsigmas_gal, nohr=True, pc_kw=dict(cmap=cmap_alpha, norm=norm_alpha))


skymap_plotter.fig.savefig('{}/skymap_gamma_galactic.png'.format(plot_dir))

#### Templates in Galactic

In [ ]:
for template_str in ['pi0', 'kra5']:
    for smearing in [0., 5, 10]:
        for coord in ['G', 'C']:
            template = get_smeared_template(template_str, smearing=smearing)
            skymap_plotter = SkymapPlotter(
                skymap=template, cmap='viridis', 
                label=r'P({}) with {:1.0f}° smearing'.format(template_str, smearing), 
                coord=coord,
            )
            skymap_plotter.draw_equator()
            skymap_plotter.fig.savefig('{}/template_{}_smearing_{:1.1f}_{}.png'.format(
                plot_dir, template_str, smearing, coord))        

In [ ]:
for ipx in np.argsort(ss_nsigmas)[::-1][:5]:
    theta, phi = hp.pix2ang(128, ipx)
    dec = np.pi/2.  - theta
    print(np.rad2deg(dec), np.rad2deg(phi))

In [ ]:
np.argsort(ss_nsigmas_cpy)[::-1]

In [ ]:
ra_deg, dec_deg = fermi_sources['Crab']
theta = np.pi/2. - np.deg2rad(dec_deg)
phi = np.deg2rad(ra_deg)
mask_crab = hp.query_disc(
    nside=128, 
    vec=hp.dir2vec(theta=theta, phi=phi), 
    radius=np.deg2rad(10),
)
ipx_crab = hp.ang2pix(128, theta=theta, phi=phi)



ss_nsigmas_cpy = np.zeros_like(ss_nsigmas)
ss_nsigmas_cpy[mask_crab] = np.array(ss_nsigmas[ipix_crab])
fig, ax, sp = plot_skymap(ss_nsigmas_cpy, label=r'$n\cdot \sigma$')

for ipx in np.argsort(ss_nsigmas_cpy)[::-1][:5]:
    theta, phi = hp.pix2ang(128, ipx)
    dec = np.pi/2.  - theta
    print('{:3.3f} {:3.3f} | {:3.3f} $n\cdot \sigma$'.format(
        np.rad2deg(phi), np.rad2deg(dec), ss_nsigmas_cpy[ipx]))
print(fermi_sources['Crab'], ss_nsigmas[ipx_crab])

#### Collect event ids for 10 most energetic events

In [ ]:
idx = np.argsort(a.bg_data.energy)[::-1][:10]
print(a.bg_data.energy[idx])

import pandas as pd

df = pd.DataFrame({
    'run': a.bg_data.run[idx],
    'event': a.bg_data.event[idx],
    'subevent': a.bg_data.subevent[idx],
})
df.to_pickle('{}/top_10E_events.pickle'.format(plot_dir))
df


In [ ]:
for gamma in [2., 2.5, 3.]:
    plt.hist(a.sig.true_energy, bins=np.logspace(1, 7, 100), histtype='step', density=True,
             weights=a.sig.oneweight * a.sig.true_energy**-gamma, label='gamma: {:3.3f}'.format(gamma))
plt.axvline(100, color='0.7', ls=':')
plt.axvline(500, color='0.7', ls='--')
plt.axvline(1000, color='0.7', ls='-')
plt.xscale('log')
plt.yscale('log')
plt.legend()


In [ ]:
from scipy import stats
s = 1.
stats.norm.sf(s), stats.norm.sf(s)*109, 1./109